In [ ]:
import Ipynb_importer
import numpy as np
import pandas as pd
from pandas import DataFrame
import utils,preprocess,merge_base
import time
import datetime
import os,gc
file_path='D:/360WiFi/'
raw_data_path='D:/360WiFi/FDDC_financial_data_20180613/'
split_file_path='D:/360WiFi/files/financial_data_new/'
input_path='D:/360WiFi/files/input/'
mid_path='D:/360WiFi/files/mid_file/'
other_path='D:/360WiFi/files/other_file/'


#资产负债表
del_col_Balance_Bank=['AE,AA,LE,LA,SEE,SEA,OTH_EFFECT_SE,OTH_EFFECT_SA,LEE,LEA']
del_col_Balance_GB=['CAE,CAA,NCAE,NCAA,AE,AA,CLE,CLA,NCLE,NCLA,LE,LA,SEE,SEA,OTH_EFFECT_SE,OTH_EFFECT_SA,LEE,LEA']
del_col_Balance_Insurance=['AE,AA,LE,LA,SEE,SEA,OTH_EFFECT_SE,OTH_EFFECT_SA,LEE,LEA']
del_col_Balance_Securities=['AE,AA,LE,LA,SEE,SEA,OTH_EFFECT_SE,OTH_EFFECT_SA,LEE,LEA']

#现金流量表
del_col_cashFlow_Bank=['SPEC_OCIF,AOCIF,SPEC_OCOF,AOCOF,ANOCF,SPEC_ICIF,AICIF,SPEC_ICOF,AICOF,ANICF,SPEC_FCIF,AFCIF,SPEC_FCOF,AFCOF,ANFCF,OTH_EFFECT_CE,ACE,OTH_EFFECT_CEI,ACEI']
del_col_cashFlow_GB=['SPEC_OCIF,AOCIF,SPEC_OCOF,AOCOF,ANOCF,SPEC_ICIF,AICIF,SPEC_ICOF,AICOF,ANICF,SPEC_FCIF,AFCIF,SPEC_FCOF,AFCOF,ANFCF,OTH_EFFECT_CE,ACE,OTH_EFFECT_CEI,ACEI']
del_col_cashFlow_Insurance=['SPEC_OCIF,AOCIF,SPEC_OCOF,AOCOF,ANOCF,SPEC_ICIF,AICIF,SPEC_ICOF,AICOF,ANICF,SPEC_FCIF,AFCIF,SPEC_FCOF,AFCOF,ANFCF,OTH_EFFECT_CE,ACE,OTH_EFFECT_CEI,ACEI']
del_col_cashFlow_Securities=['SPEC_OCIF,AOCIF,SPEC_OCOF,AOCOF,ANOCF,SPEC_ICIF,AICIF,SPEC_ICOF,AICOF,ANICF,SPEC_FCIF,AFCIF,SPEC_FCOF,AFCOF,ANFCF,OTH_EFFECT_CE,ACE,OTH_EFFECT_CEI,ACEI']


#利润表
del_col_Income_Bank=['SPEC_OR,AOR,SPEC_OC,AOC,OTH_EFFECT_OP,AE_EFFECT_OP,OTH_EFFECT_TP,AE_EFFECT_TP,OTH_EFFECT_NP,AE_EFFECT_NP,OTH_EFFECT_NPP,AE_EFFECT_NPP,OTH_EFFECT_CI,AE_EFFECT_CI,OTH_EFFECT_PCI,AE_EFFECT_PCI']
del_col_Income_GB=['SPEC_TOR,ATOR,SPEC_TOC,ATOC,OTH_EFFECT_OP,AE_EFFECT_OP,OTH_EFFECT_TP,AE_EFFECT_TP,OTH_EFFECT_NP,AE_EFFECT_NP,OTH_EFFECT_NPP,AE_EFFECT_NPP,OTH_EFFECT_CI,AE_EFFECT_CI,OTH_EFFECT_PCI,AE_EFFECT_PCI']
del_col_Income_Insurance=['SPEC_OR,AOR,SPEC_OC,AOC,OTH_EFFECT_OP,AE_EFFECT_OP,OTH_EFFECT_TP,AE_EFFECT_TP,OTH_EFFECT_NP,AE_EFFECT_NP,OTH_EFFECT_NPP,AE_EFFECT_NPP,OTH_EFFECT_CI,AE_EFFECT_CI,OTH_EFFECT_PCI,AE_EFFECT_PCI']
del_col_Income_Securities=['SPEC_OR,AOR,SPEC_OC,AOC,OTH_EFFECT_OP,AE_EFFECT_OP,OTH_EFFECT_TP,AE_EFFECT_TP,OTH_EFFECT_NP,AE_EFFECT_NP,OTH_EFFECT_NPP,AE_EFFECT_NPP,OTH_EFFECT_CI,AE_EFFECT_CI,OTH_EFFECT_PCI,AE_EFFECT_PCI']



del_list=[del_col_Balance_Bank,del_col_Balance_GB,del_col_Balance_Insurance,del_col_Balance_Securities,\
         del_col_cashFlow_Bank,del_col_cashFlow_GB,del_col_cashFlow_Insurance,del_col_cashFlow_Securities,\
        del_col_Income_Bank,del_col_Income_GB,del_col_Income_Insurance,del_col_Income_Securities]

#preprocess.preprocess(del_list)

In [ ]:
##探究主表的列类型

A=pd.read_csv(mid_path+'table_6_file.csv')
#A.columns[A.columns.str.contains('REVE')]#列中包含某字符的列名
#A.get_dtype_counts() 列的类型计数

#A.infer_objects().dtypes#具体到每列的类型
#t#[t=='object']


'''
注意这之后，先把全为空列删除，进行完分组训和划分训练集之后，还要再删的列：year，month，class_chocie

'''


class_chocie='classifi_by_14L1'

oht_col=['TICKER_SYMBOL','FISCAL_PERIOD','industrySymbol_level_1','industrySymbol_level_2','industrySymbol_level_1',\
        'indexSymbol_level_1','indexSymbol_level_2','indexSymbol_level_3',\
        ]

del_col=['PARTY_ID','EXCHANGE_CD', 'PUBLISH_DATE','END_DATE_REP',\
         'END_DATE', 'REPORT_TYPE','MERGED_FLAG', 'category',\
        'industryName_level_1','industryName_level_2','industryName_level_3','isNew','Mid_L2','Mid_L3',\
         'industryID_level_1','industryID_level_2','industryID_level_3',\
         'PUBLISH_DATE_y','PUBLISH_DATE_x','END_DATE_REP_x','END_DATE_REP_y'
        ]

def select_table(A,class_chocie,del_col):

    class_col=['classifi_by_14L1','classifi_by_14L2','classifi_by_14L3']
    class_col.remove(class_chocie)

    del_col.extend(class_col)
    del_col
    
    A.dropna(axis=1,how='all',inplace=True) 
    A.drop(del_col,axis=1,inplace=True)

    ##----检查 全为空或者0的列----------------
    print('--------------------------------')
    print('Now eheck the [none,0] col:\n')
    check_col=A.sum()
    print(check_col[check_col==0].index)
    print('\n')

    for none_col in check_col[check_col==0].index:
        del A[none_col]
        print('successfully del:',none_col)
    print('--------------------------------')


    ##----检查类型异常的列----------------
    print('Now check the tpyes of col:\n')
    print(A.get_dtype_counts())
    print('--------------------------------')
    print('The int col are:\n')
    int_col=A.infer_objects().dtypes
    print(int_col[int_col=='int64'])
    print('--------------------------------')
    
    return 
select_table(A,class_chocie,del_col)
#A.infer_objects().dtypes

In [ ]:
import xgboost as xgb

In [ ]:
T_new.industrySymbol_level_1.unique()

In [ ]:
# def compare(df1,df2):
#     '''
#     比较表间ticker异同
    
#     '''
#     print(len(df1.TICKER_SYMBOL.unique()))
#     print(len(df2.TICKER_SYMBOL.unique()))
#     print(set(df1.TICKER_SYMBOL.unique())-set(df2.TICKER_SYMBOL.unique()))
#     print(set(df2.TICKER_SYMBOL.unique())-set(df1.TICKER_SYMBOL.unique()))
    
    
# compare(balance_sheet_Securities_old,income_sheet_Securities_old)

In [ ]:


###-------生成器读取文件-----------------
# def loading_inputs():
#     files= os.listdir(input_path+'/raw_data/')
#     for file in files:
#         print('reading file:%s'%file)
#         feature = pd.read_csv(input_path+'/raw_data/'+file)
#         yield feature
# T=[]
# for item in loading_inputs():
#     T.append(item)


###-------读取文件---------------------
def loading_inputs():
    files= os.listdir(input_path+'/raw_data/')
    DF=[]
    for file in files:
        print('reading file:%s'%file)
        feature = pd.read_csv(input_path+'/raw_data/'+file)
        DF.append(feature)
    
    return DF


def preprocess_common(del_list):
    '''
    输入是要删除的列，输出是删除指定列并去重的df
    
    '''
    print('----------------------------------------------------')
    print('Starting preprocessing....\n')
    print('=======================================================')
    print('loading file...')
    T=loading_inputs()

    ##--------预处理:删列操作-------------------
    print('=======================================================')
    print('del some cols...')
    for i in range(12):
        print('del col of df')
        try:
            T[i]=T[i].drop(del_list[i][0].split(','),axis=1)
        except Exception as err:
            print(err)

    ##-----------单表去重操作----------------------------

    print('single sheet drop_duplicates...')

    for i in range(12):
        T[i]=utils.date_time_index(T[i]).drop_duplicates(subset=['TICKER_SYMBOL','END_DATE'], keep='last')

    return T

def preprocess_normal(T):
    '''
    处理正常的ticker,首先对三大表内部各自合四表，最后合三大表
    
    '''
    print('=======================================================')
    print('preprosessing the normal ticker...')
    [balance_sheet_Bank_old,balance_sheet_GB_old,balance_sheet_Insurance_old,balance_sheet_Securities_old,\
    cashFlow_sheet_Bank_old,cashFlow_sheet_GB_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_Securities_old,\
    income_sheet_Bank_old,income_sheet_GB_old,income_sheet_Insurance_old,income_sheet_Securities_old]=T
    del T
    gc.collect()

    ###----------------预处理：添加类别列，找公共columns-----------------------
    ##添加类别列

    balance_sheet_Bank_old['category']='Bank'
    cashFlow_sheet_Bank_old['category']='Bank'
    income_sheet_Bank_old['category']='Bank'

    balance_sheet_Securities_old['category']='Securities'
    cashFlow_sheet_Securities_old['category']='Securities'
    income_sheet_Securities_old['category']='Securities'

    balance_sheet_Insurance_old['category']='Insurance'
    cashFlow_sheet_Insurance_old['category']='Insurance'
    income_sheet_Insurance_old['category']='Insurance'

    balance_sheet_GB_old['category']='GB'
    cashFlow_sheet_GB_old['category']='GB'
    income_sheet_GB_old['category']='GB'


    ##查找columns公共部分
        #资产负债
    A1=balance_sheet_Bank_old.columns
    B1=balance_sheet_Securities_old.columns
    C1=balance_sheet_Insurance_old.columns
    D1=balance_sheet_GB_old.columns
        #现金流量
    A2=cashFlow_sheet_Bank_old.columns
    B2=cashFlow_sheet_Securities_old.columns
    C2=cashFlow_sheet_Insurance_old.columns
    D2=cashFlow_sheet_GB_old.columns
        #利润表
    A3=income_sheet_Bank_old.columns
    B3=income_sheet_Securities_old.columns
    C3=income_sheet_Insurance_old.columns
    D3=income_sheet_GB_old.columns


    #确定最终放在表首的几列
    common=['INDEX','END_DATE', 'END_DATE_REP','EXCHANGE_CD','FISCAL_PERIOD','MERGED_FLAG','PARTY_ID',\
     'PUBLISH_DATE','REPORT_TYPE','TICKER_SYMBOL','category']



    ##分别对三表，找出每个表中四个类别公共部分

    commom_balance=set(C1)&set(D1)&set(A1)&set(B1)#-set(common)
    commom_cash=set(C2)&set(D2)&set(A2)&set(B2)#-set(common)
    commom_income=set(C3)&set(D3)&set(A3)&set(B3)#-set(common)

    commom_balance=list(commom_balance)
    commom_cash=list(commom_cash)
    commom_income=list(commom_income)

    ##列出几个特殊ticker,暂时去掉

    del_ticker=[563, 627, 712, 750, 776, 987, 2673, 600291, 600816]

    T1=[balance_sheet_Bank_old,balance_sheet_GB_old,balance_sheet_Insurance_old,balance_sheet_Securities_old,\
     cashFlow_sheet_Bank_old,cashFlow_sheet_GB_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_Securities_old,\
    income_sheet_Bank_old,income_sheet_GB_old,income_sheet_Insurance_old,income_sheet_Securities_old]

    for i in range(12):
        #选出不在列表中出现的ticker的数据
        IN=np.logical_not(T1[i].TICKER_SYMBOL.isin(del_ticker))
        T1[i]=T1[i][IN]

    [balance_sheet_Bank_old,balance_sheet_GB_old,balance_sheet_Insurance_old,balance_sheet_Securities_old,\
     cashFlow_sheet_Bank_old,cashFlow_sheet_GB_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_Securities_old,\
    income_sheet_Bank_old,income_sheet_GB_old,income_sheet_Insurance_old,income_sheet_Securities_old]=T1
    del T1
    gc.collect()    


    ##--------------------资产负债表合表------------------------------

    print('merging Balance sheet.....\n')

    #提取公共部分并concat

    concat_list_balance=[balance_sheet_Bank_old[commom_balance],balance_sheet_Securities_old[commom_balance],\
            balance_sheet_Insurance_old[commom_balance],balance_sheet_GB_old[commom_balance]]

    df_balance_common=pd.concat(concat_list_balance)

    #将公共部分统一加后缀
    old_common_column_balance=list(set(commom_balance)-set(common))
    new_common_column_balance=[x+'_commom_balance' for x in old_common_column_balance]

    df_balance_common.rename(columns=dict(zip(old_common_column_balance,new_common_column_balance)),inplace=True)
    df_balance_common


    ##依次提取并merge四表剩余部分

    balance_df=[balance_sheet_Bank_old,balance_sheet_Securities_old,balance_sheet_Insurance_old,balance_sheet_GB_old]
    name=['_balance_Bank','_balance_Securities','_balance_Insurance','_balance_GB']
    balance_list=[]

    for a,b in zip(balance_df,name):    
        balance_bank_columns=list(set(a.columns)-set(commom_balance))
        balance_bank_columns.extend(common)
        t=set(balance_bank_columns)-set(common)
        balance_bank_columns_new=[x+ b for x in t]
        balance_sheet_Bank_old_rest=a[balance_bank_columns]
        balance_sheet_Bank_old_rest.rename(columns=dict(zip(t,balance_bank_columns_new)),inplace=True)

        balance_list.append(balance_sheet_Bank_old_rest)
    '''
     merge有一个大bug,左连接时，merge(a,b),如果a和b都有中有多行相同，如a中有 ee行 = ff行  ，而b中也有  cc行= dd 行  ，
     且 ee=cc=ff=dd就会出现最终merge结果条数>a；或者说b中有两条与a中某条对应，亦是如此.
  
    '''   

    merge1=pd.merge(df_balance_common,balance_list[0],how='left',on=common)
    merge2=pd.merge(merge1,balance_list[1],how='left',on=common)
    merge3=pd.merge(merge2,balance_list[2],how='left',on=common)
    merge_balance=pd.merge(merge3,balance_list[3],how='left',on=common)

    merge_balance

    ##--------------------现金流量表合表------------------------------
    print('merging CashFlow sheet.....\n')

    #提取公共部分并concat

    concat_list_cashFlow=[cashFlow_sheet_Bank_old[commom_cash],cashFlow_sheet_Securities_old[commom_cash],\
    cashFlow_sheet_Insurance_old[commom_cash],cashFlow_sheet_GB_old[commom_cash]]
    df_cashFlow_common=pd.concat(concat_list_cashFlow)

    df_cashFlow_common

    #将公共部分统一加后缀
    old_common_column_cash=list(set(commom_cash)-set(common))
    new_common_column_cash=[x+'_commom_cash' for x in old_common_column_cash]
    df_cashFlow_common.rename(columns=dict(zip(old_common_column_cash,new_common_column_cash)),inplace=True)

    df_cashFlow_common


    ##依次提取并merge四表剩余部分

    cash_df=[cashFlow_sheet_Bank_old,cashFlow_sheet_Securities_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_GB_old]
    name=['_cashFlow_Bank','_cashFlow_Securities','_cashFlow_Insurance','_cashFlow_GB']
    cash_list=[]

    for a,b in zip(cash_df,name):    
        cash_bank_columns=list(set(a.columns)-set(commom_cash))
        cash_bank_columns.extend(common)
        t=set(cash_bank_columns)-set(common)
        cash_bank_columns_new=[x+ b for x in t]
        cash_sheet_Bank_old_rest=a[cash_bank_columns]
        cash_sheet_Bank_old_rest.rename(columns=dict(zip(t,cash_bank_columns_new)),inplace=True)

        cash_list.append(cash_sheet_Bank_old_rest)


    merge1=pd.merge(df_cashFlow_common,cash_list[0],how='left',on=common)
    merge2=pd.merge(merge1,cash_list[1],how='left',on=common)
    merge3=pd.merge(merge2,cash_list[2],how='left',on=common)
    merge_cash=pd.merge(merge3,cash_list[3],how='left',on=common)
    merge_cash

    ##--------------------利润表合表------------------------------
    print('merging Income sheet.....\n')

    #提取公共部分并concat

    concat_list_income=[income_sheet_Bank_old[commom_income],income_sheet_Securities_old[commom_income],\
    income_sheet_Insurance_old[commom_income],income_sheet_GB_old[commom_income]]
    df_income_common=pd.concat(concat_list_income)

    df_income_common

    #将公共部分统一加后缀
    old_common_column_income=list(set(commom_income)-set(common))
    new_common_column_income=[x+'_commom_income' for x in old_common_column_income]
    df_income_common.rename(columns=dict(zip(old_common_column_income,new_common_column_income)),inplace=True)

    df_income_common


    ##依次提取并merge四表剩余部分

    income_df=[income_sheet_Bank_old,income_sheet_Securities_old,income_sheet_Insurance_old,income_sheet_GB_old]
    name=['_income_Bank','_income_Securities','_income_Insurance','_income_GB']
    income_list=[]

    for a,b in zip(income_df,name):    
        income_bank_columns=list(set(a.columns)-set(commom_income))
        income_bank_columns.extend(common)
        t=set(income_bank_columns)-set(common)
        income_bank_columns_new=[x+ b for x in t]
        income_sheet_Bank_old_rest=a[income_bank_columns]
        income_sheet_Bank_old_rest.rename(columns=dict(zip(t,income_bank_columns_new)),inplace=True)

        income_list.append(income_sheet_Bank_old_rest)

    merge1=pd.merge(df_income_common,income_list[0],how='left',on=common)
    merge2=pd.merge(merge1,income_list[1],how='left',on=common)
    merge3=pd.merge(merge2,income_list[2],how='left',on=common)
    merge_income=pd.merge(merge3,income_list[3],how='left',on=common)
    merge_income


    ##----------------合三表-------------------------

    print('merging final sheet.....\n')
    merge_balance=merge_balance.drop(['INDEX'],axis=1)
    merge_cash=merge_cash.drop(['INDEX'],axis=1)
    merge_income=merge_income.drop(['INDEX'],axis=1)
    merge_income

    M1=pd.merge(merge_balance,merge_cash,how='outer',on=['END_DATE','EXCHANGE_CD','FISCAL_PERIOD','MERGED_FLAG','PARTY_ID',\
                                         'REPORT_TYPE','TICKER_SYMBOL','category'])##index,publist_date h和 enddate_rep应该删掉！
    M2=pd.merge(M1,merge_income,how='outer',on=['END_DATE','EXCHANGE_CD','FISCAL_PERIOD','MERGED_FLAG','PARTY_ID',\
                                         'REPORT_TYPE','TICKER_SYMBOL','category'])
    
    
    print('preprosessing the normal ticker done!!!...\n')
    print('=======================================================')
    return M2

def preprocess_special(T):
    
    '''
    处理有冲突的ticker,首先对三大表内部各自合四表，最后合三大表
    
    '''
    print('preprosessing the special ticker...')
    [balance_sheet_Bank_old,balance_sheet_GB_old,balance_sheet_Insurance_old,balance_sheet_Securities_old,\
     cashFlow_sheet_Bank_old,cashFlow_sheet_GB_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_Securities_old,\
    income_sheet_Bank_old,income_sheet_GB_old,income_sheet_Insurance_old,income_sheet_Securities_old]=T
    del T
    gc.collect()

    ###----------------预处理：添加类别列，找公共columns-----------------------
    ##添加类别列

    balance_sheet_Bank_old['category']='SSR'
    cashFlow_sheet_Bank_old['category']='SSR'
    income_sheet_Bank_old['category']='SSR'

    balance_sheet_Securities_old['category']='SSR'
    cashFlow_sheet_Securities_old['category']='SSR'
    income_sheet_Securities_old['category']='SSR'

    balance_sheet_Insurance_old['category']='SSR'
    cashFlow_sheet_Insurance_old['category']='SSR'
    income_sheet_Insurance_old['category']='SSR'

    balance_sheet_GB_old['category']='SSR'
    cashFlow_sheet_GB_old['category']='SSR'
    income_sheet_GB_old['category']='SSR'



    ##查找columns公共部分
        #资产负债
    A1=balance_sheet_Bank_old.columns
    B1=balance_sheet_Securities_old.columns
    C1=balance_sheet_Insurance_old.columns
    D1=balance_sheet_GB_old.columns
        #现金流量
    A2=cashFlow_sheet_Bank_old.columns
    B2=cashFlow_sheet_Securities_old.columns
    C2=cashFlow_sheet_Insurance_old.columns
    D2=cashFlow_sheet_GB_old.columns
        #利润表
    A3=income_sheet_Bank_old.columns
    B3=income_sheet_Securities_old.columns
    C3=income_sheet_Insurance_old.columns
    D3=income_sheet_GB_old.columns


    #确定最终放在表首的几列
    common=['INDEX','END_DATE', 'END_DATE_REP','EXCHANGE_CD','FISCAL_PERIOD','MERGED_FLAG','PARTY_ID',\
     'PUBLISH_DATE','REPORT_TYPE','TICKER_SYMBOL','category']


    ##分别对三表，找出每个表中四个类别公共部分

    commom_balance=set(C1)&set(D1)&set(A1)&set(B1)#-set(common)
    commom_cash=set(C2)&set(D2)&set(A2)&set(B2)#-set(common)
    commom_income=set(C3)&set(D3)&set(A3)&set(B3)#-set(common)

    commom_balance=list(commom_balance)
    commom_cash=list(commom_cash)
    commom_income=list(commom_income)



    ##选出ticker条目
    del_ticker=[563, 627, 712, 750, 776, 987, 2673, 600291, 600816]

    T1=[balance_sheet_Bank_old,balance_sheet_GB_old,balance_sheet_Insurance_old,balance_sheet_Securities_old,\
     cashFlow_sheet_Bank_old,cashFlow_sheet_GB_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_Securities_old,\
    income_sheet_Bank_old,income_sheet_GB_old,income_sheet_Insurance_old,income_sheet_Securities_old]

    for i in range(12):
        T1[i]=T1[i][T1[i].TICKER_SYMBOL.isin(del_ticker)]

    [balance_sheet_Bank_old,balance_sheet_GB_old,balance_sheet_Insurance_old,balance_sheet_Securities_old,\
     cashFlow_sheet_Bank_old,cashFlow_sheet_GB_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_Securities_old,\
    income_sheet_Bank_old,income_sheet_GB_old,income_sheet_Insurance_old,income_sheet_Securities_old]=T1

    del T1
    gc.collect()


    ##--------------------资产负债表合表------------------------------

    print('merging Balance sheet.....\n')


    #提取公共部分并concat


    concat_list_balance=[balance_sheet_Bank_old[commom_balance],balance_sheet_Securities_old[commom_balance],\
            balance_sheet_Insurance_old[commom_balance],balance_sheet_GB_old[commom_balance]]

    df_balance_common=pd.concat(concat_list_balance)

    #将公共部分统一加后缀
    old_common_column_balance=list(set(commom_balance)-set(common))
    new_common_column_balance=[x+'_commom_balance' for x in old_common_column_balance]

    df_balance_common.rename(columns=dict(zip(old_common_column_balance,new_common_column_balance)),inplace=True)
    df_balance_common



    ##依次提取并merge四表剩余部分

    balance_df=[balance_sheet_Bank_old,balance_sheet_Securities_old,balance_sheet_Insurance_old,balance_sheet_GB_old]
    name=['_balance_Bank','_balance_Securities','_balance_Insurance','_balance_GB']
    balance_list=[]

    for a,b in zip(balance_df,name):    
        balance_bank_columns=list(set(a.columns)-set(commom_balance))
        balance_bank_columns.extend(common)
        t=set(balance_bank_columns)-set(common)
        balance_bank_columns_new=[x+ b for x in t]
        balance_sheet_Bank_old_rest=a[balance_bank_columns]
        balance_sheet_Bank_old_rest.rename(columns=dict(zip(t,balance_bank_columns_new)),inplace=True)

        balance_list.append(balance_sheet_Bank_old_rest)

    ##再次去重    
    df_balance_common=df_balance_common.sort_values(by=['TICKER_SYMBOL','END_DATE','PUBLISH_DATE','END_DATE_REP'])
    df_balance_common.drop_duplicates(subset=['TICKER_SYMBOL','END_DATE'],keep='last',inplace=True)                                                    


    merge1=pd.merge(df_balance_common,balance_list[0],how='left',on=common)
    merge2=pd.merge(merge1,balance_list[1],how='left',on=common)
    merge3=pd.merge(merge2,balance_list[2],how='left',on=common)
    merge_balance=pd.merge(merge3,balance_list[3],how='left',on=common)

    merge_balance


#     ##-------------检查concat后是否重复-------------------------

#     t=df_balance_common.groupby(['TICKER_SYMBOL','END_DATE']).size()
#     t=t[t!=1].reset_index()#.to_csv(file_path+'Balance_conflict.csv',index=False)


#     def check_ssr(df):
#     '''
#     找出concat后，重复ticker-enddate，的条目数据，公共部分columns是否一致
#     '''
#         t=df.groupby(['TICKER_SYMBOL','END_DATE']).size()
#         t=t[t!=1].reset_index()
#         t.END_DATE=t.END_DATE.apply(lambda x:datetime.datetime.strftime(x,'%Y-%m-%d'))
#         for m,n in zip(t.TICKER_SYMBOL,t.END_DATE):
#             a=list(df.columns)
#             a.remove('PUBLISH_DATE')
#             a.remove('END_DATE_REP')
#             a.remove('INDEX')
#             if len(df[(df.TICKER_SYMBOL==m)&(df.END_DATE==n)].drop_duplicates(subset=a))==1:
#                 print(m,n)#to_csv(file_path+'123.csv',index=False)#.drop_duplicates(subset=a)



    ##--------------------现金流量表合表------------------------------
    print('merging CashFlow sheet.....\n')

    #提取公共部分并concat

    concat_list_cashFlow=[cashFlow_sheet_Bank_old[commom_cash],cashFlow_sheet_Securities_old[commom_cash],\
    cashFlow_sheet_Insurance_old[commom_cash],cashFlow_sheet_GB_old[commom_cash]]
    df_cashFlow_common=pd.concat(concat_list_cashFlow)

    df_cashFlow_common

    #将公共部分统一加后缀
    old_common_column_cash=list(set(commom_cash)-set(common))
    new_common_column_cash=[x+'_commom_cash' for x in old_common_column_cash]
    df_cashFlow_common.rename(columns=dict(zip(old_common_column_cash,new_common_column_cash)),inplace=True)

    df_cashFlow_common



    ##依次提取并merge四表剩余部分

    cash_df=[cashFlow_sheet_Bank_old,cashFlow_sheet_Securities_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_GB_old]
    name=['_cashFlow_Bank','_cashFlow_Securities','_cashFlow_Insurance','_cashFlow_GB']
    cash_list=[]

    for a,b in zip(cash_df,name):    
        cash_bank_columns=list(set(a.columns)-set(commom_cash))
        cash_bank_columns.extend(common)
        t=set(cash_bank_columns)-set(common)
        cash_bank_columns_new=[x+ b for x in t]
        cash_sheet_Bank_old_rest=a[cash_bank_columns]
        cash_sheet_Bank_old_rest.rename(columns=dict(zip(t,cash_bank_columns_new)),inplace=True)

        cash_list.append(cash_sheet_Bank_old_rest)

    ##再次去重
    df_cashFlow_common=df_cashFlow_common.sort_values(by=['TICKER_SYMBOL','END_DATE','PUBLISH_DATE','END_DATE_REP'])
    df_cashFlow_common.drop_duplicates(subset=['TICKER_SYMBOL','END_DATE'],keep='last',inplace=True)       

    merge1=pd.merge(df_cashFlow_common,cash_list[0],how='left',on=common)
    merge2=pd.merge(merge1,cash_list[1],how='left',on=common)
    merge3=pd.merge(merge2,cash_list[2],how='left',on=common)
    merge_cash=pd.merge(merge3,cash_list[3],how='left',on=common)
    merge_cash


    ##--------------------利润表合表------------------------------
    print('merging Income sheet.....\n')

    #提取公共部分并concat

    concat_list_income=[income_sheet_Bank_old[commom_income],income_sheet_Securities_old[commom_income],\
    income_sheet_Insurance_old[commom_income],income_sheet_GB_old[commom_income]]
    df_income_common=pd.concat(concat_list_income)

    df_income_common

    #将公共部分统一加后缀
    old_common_column_income=list(set(commom_income)-set(common))
    new_common_column_income=[x+'_commom_income' for x in old_common_column_income]
    df_income_common.rename(columns=dict(zip(old_common_column_income,new_common_column_income)),inplace=True)

    df_income_common


    ##依次提取并merge四表剩余部分

    income_df=[income_sheet_Bank_old,income_sheet_Securities_old,income_sheet_Insurance_old,income_sheet_GB_old]
    name=['_income_Bank','_income_Securities','_income_Insurance','_income_GB']
    income_list=[]

    for a,b in zip(income_df,name):    
        income_bank_columns=list(set(a.columns)-set(commom_income))
        income_bank_columns.extend(common)
        t=set(income_bank_columns)-set(common)
        income_bank_columns_new=[x+ b for x in t]
        income_sheet_Bank_old_rest=a[income_bank_columns]
        income_sheet_Bank_old_rest.rename(columns=dict(zip(t,income_bank_columns_new)),inplace=True)

        income_list.append(income_sheet_Bank_old_rest)


    ##再次去重
    df_income_common=df_income_common.sort_values(by=['TICKER_SYMBOL','END_DATE','PUBLISH_DATE','END_DATE_REP'])
    df_income_common.drop_duplicates(subset=['TICKER_SYMBOL','END_DATE'],keep='last',inplace=True) 


    merge1=pd.merge(df_income_common,income_list[0],how='left',on=common)
    merge2=pd.merge(merge1,income_list[1],how='left',on=common)
    merge3=pd.merge(merge2,income_list[2],how='left',on=common)
    merge_income=pd.merge(merge3,income_list[3],how='left',on=common)
    merge_income




    ##----------------合三表-------------------------

    print('merging final sheet.....\n')
    merge_balance=merge_balance.drop(['INDEX'],axis=1)
    merge_cash=merge_cash.drop(['INDEX'],axis=1)
    merge_income=merge_income.drop(['INDEX'],axis=1)
    merge_income

    M1=pd.merge(merge_balance,merge_cash,how='outer',on=['END_DATE','EXCHANGE_CD','FISCAL_PERIOD','MERGED_FLAG','PARTY_ID',\
                                         'REPORT_TYPE','TICKER_SYMBOL','category'])
    M2=pd.merge(M1,merge_income,how='outer',on=['END_DATE','EXCHANGE_CD','FISCAL_PERIOD','MERGED_FLAG','PARTY_ID',\
                                         'REPORT_TYPE','TICKER_SYMBOL','category'])
    M2
    
    print('preprosessing the special ticker done!!!...')
    print('=======================================================')
    
    
    return M2



def preprocess(del_list):
    '''
    生成财务合并表：merge_file.csv
    '''
    base_file=preprocess_common(del_list)
    file_normal=preprocess_normal(base_file)
    file_special=preprocess_special(base_file)
    
    merge_file=pd.concat([file_normal,file_special])
    
    common=['PARTY_ID','TICKER_SYMBOL', 'EXCHANGE_CD','PUBLISH_DATE','END_DATE_REP','END_DATE','FISCAL_PERIOD',\
        'REPORT_TYPE', 'MERGED_FLAG', 'category']
    
    merge_file=merge_file[common+list(set(merge_file.columns)-set(common))]
    
    
    print('the shape of normal ticker file:',file_normal.shape)
    print('the shape of special ticker file:',file_special.shape)
    print('the shape of merge file:',merge_file.shape)
    
    merge_file.to_csv(mid_path+'merge_file.csv',index=False)
    

    
    return merge_file
    
# ##再次check merge_balance,merge_cash，merge_income，M2
# ###是否有'TICKER_SYMBOL'-'END_DATE'重复项
# '''
# 也就是比较drop前后长度是否相等，若相等，则正确
# '''
# def check_dup(DF):
#     print(DF.shape)
#     print(DF.drop_duplicates(subset=['TICKER_SYMBOL','END_DATE'], keep='last').shape)

# check_dup(M2)
T=preprocess(del_list)

In [ ]:
common=['PARTY_ID','TICKER_SYMBOL', 'EXCHANGE_CD','PUBLISH_DATE','END_DATE_REP','END_DATE','FISCAL_PERIOD',\
        'REPORT_TYPE', 'MERGED_FLAG', 'category']


T[common+list(set(T.columns)-set(common))]


In [ ]:
M2[M2.TICKER_SYMBOL==30][['TICKER_SYMBOL','END_DATE','REVENUE_commom_income']].sort_values(by=['END_DATE'])

In [ ]:
#t=M2.columns.str.contains('REVENUE')
#M2.columns[t]
submit=pd.read_csv(file_path+'sumbit_v1.csv')
base=submit[['TICKER_SYMBOL']]
base

time_range=['2009-03-31', '2009-06-30', '2009-09-30', '2009-12-31',
       '2010-03-31', '2010-06-30', '2010-09-30', '2010-12-31',
       '2011-03-31', '2011-06-30', '2011-09-30', '2011-12-31',
       '2012-03-31', '2012-06-30', '2012-09-30', '2012-12-31',
       '2013-03-31', '2013-06-30', '2013-09-30', '2013-12-31',
       '2014-03-31', '2014-06-30', '2014-09-30', '2014-12-31',
       '2015-03-31', '2015-06-30', '2015-09-30', '2015-12-31',
       '2016-03-31', '2016-06-30', '2016-09-30', '2016-12-31',
       '2017-03-31', '2017-06-30', '2017-09-30', '2017-12-31',
       '2018-03-31']

for month in time_range:
    
    REVENUE_df=M2[['END_DATE','TICKER_SYMBOL','REVENUE_commom_income']]
    t=REVENUE_df[REVENUE_df.END_DATE==month].sort_values(by=['END_DATE']).dropna()
    t.rename(columns={'REVENUE_commom_income':month},inplace=True)
    base=pd.merge(base,t[['TICKER_SYMBOL',month]],how='left',on=['TICKER_SYMBOL'])
    
base.to_csv(file_path+'ticker_REVENUE_update.csv',index=False)
# q=REVENUE_df[REVENUE_df.TICKER_SYMBOL==1].sort_values(by=['END_DATE']).dropna()
# q['END_DATE']=q['END_DATE'].apply(lambda x:datetime.datetime.strftime(x,'%Y-%m-%d'))
# q['END_DATE'].values

In [ ]:

    ##1.market data 检查value 和申万知否一致
    
#market_data=pd.read_excel(raw_data_path+'[New] Market Data_20180613.xlsx')
#market_data.TICKER_SYMBOL.unique().shape#(3675,)
# test_180330=market_data[market_data['END_DATE_']=='2018-03-30'][['TICKER_SYMBOL','MARKET_VALUE']]
# check_180830=pd.read_csv(file_path+'Uer_market_2018330.csv')
# check_180830=check_180830[['ticker','negMarketValue','marketValue']].rename(columns={'ticker':'TICKER_SYMBOL'})
# pd.merge(test_180330,check_180830,how='left',on=['TICKER_SYMBOL']).to_csv(file_path+'check_market_values.csv',index=False)
#================================================================
    ##2.从三表中读取12表

# balance_sheet_Bank,balance_sheet_Securities, balance_sheet_Insurance,balance_sheet_GB,\
# cashFlow_sheet_Bank,cashFlow_sheet_Securities,cashFlow_sheet_Insurance,cashFlow_sheet_GB,\
# income_sheet_Bank,income_sheet_Securities,income_sheet_Insurance,income_sheet_GB=utils.loadingFile(0)

#================================================================
    ##3.将12个表写到csv中，方便今后读取

# file_name=['balance_sheet_Bank','balance_sheet_Securities', 'balance_sheet_Insurance','balance_sheet_GB',\
# 'cashFlow_sheet_Bank','cashFlow_sheet_Securities','cashFlow_sheet_Insurance','cashFlow_sheet_GB',\
# 'income_sheet_Bank','income_sheet_Securities','income_sheet_Insurance','income_sheet_GB']

# file_df=[balance_sheet_Bank,balance_sheet_Securities, balance_sheet_Insurance,balance_sheet_GB,\
# cashFlow_sheet_Bank,cashFlow_sheet_Securities,cashFlow_sheet_Insurance,cashFlow_sheet_GB,\
# income_sheet_Bank,income_sheet_Securities,income_sheet_Insurance,income_sheet_GB]

# for df,name in zip(file_df,file_name):
#     df.to_csv(file_path+'files/financial_data_new/%s.csv'%name)
#============================================================================
    ## 4.在csv文件中补全00000

# A=pd.read_csv(file_path+'sumbit_5_31.csv',encoding='gbk')
# A.TICKER_SYMBOL=A.TICKER_SYMBOL.apply(lambda x:'\t%06d'%x)
# A.to_csv(file_path+'523.csv',index=False)

# A=pd.read_csv(file_path+'sumbit_5_31.csv',encoding='gbk')
# A.TICKER_SYMBOL=A.TICKER_SYMBOL.apply(lambda x:'%06d'%x)
# #A.to_csv(file_path+'523.csv',index=False)
# for a in A.TICKER_SYMBOL:
#     print(a)

#==============================================================
    ## 5.补全下载数据的ticker
# file_df=pd.read_csv(file_path+'cashflow_daily_all.csv')
# #file_df=pd.read_hdf(file_path+'CAI.h5',key='data')
# ticker_df=pd.read_csv(file_path+'ticker_list_new.csv')
# file_tickers=file_df.ticker.unique()
# ticker_tickers=ticker_df['0'].values
# dfii_list=[x for x in ticker_tickers if x not in file_tickers]
# DataFrame(dfii_list).to_csv(file_path+'DIFF-个股资金日流向.csv',index=False)


In [ ]:

#===================================================================================    
    ##三表ticker检查
'''
compared with 529data:add 5 more tickers
 array([300747], dtype=int64),
 array([601066], dtype=int64),
 array([603650], dtype=int64),
 array([603666], dtype=int64),
 array([603693], dtype=int64)]

'''


# Balance=[balance_sheet_Bank_old,balance_sheet_Securities_old,balance_sheet_Insurance_old,balance_sheet_GB_old]
# del balance_sheet_Bank_old,balance_sheet_Securities_old,balance_sheet_Insurance_old,balance_sheet_GB_old
# Balance

# Balance_df=pd.concat(Balance)
# Balance_df.TICKER_SYMBOL.unique()#3551

# Cash=[cashFlow_sheet_Bank_old,cashFlow_sheet_Securities_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_GB_old]
# del cashFlow_sheet_Bank_old,cashFlow_sheet_Securities_old,cashFlow_sheet_Insurance_old,cashFlow_sheet_GB_old
# Cash

# Cash_df=pd.concat(Cash)
# Cash_df.TICKER_SYMBOL.unique()#3551

# Income=[income_sheet_Bank_old,income_sheet_Securities_old,income_sheet_Insurance_old,income_sheet_GB_old]
# del income_sheet_Bank_old,income_sheet_Securities_old,income_sheet_Insurance_old,income_sheet_GB_old
# Income

# Income_df=pd.concat(Income)
# Income_df.TICKER_SYMBOL.unique()#3551



# print('the shape of Balance_df is',Balance_df.TICKER_SYMBOL.unique().shape)#(3556,)
# print('the shape of Cash_df is',Cash_df.TICKER_SYMBOL.unique().shape)#(3556,)
# print('the shape of Income_df is',Income_df.TICKER_SYMBOL.unique().shape)#()(3556,)


# tiker_list=pd.read_csv(file_path+'ticker_list.csv')
# tiker_list['0'].unique()


# #比较三大表中ticker是否相同
# A1=set(Balance_df.TICKER_SYMBOL.unique())
# A2=set(Cash_df.TICKER_SYMBOL.unique())
# A3=set(Income_df.TICKER_SYMBOL.unique())

# print('A1==A2',A1==A2)
# print('A1==A3',A1==A3)
# print('A2==A3',A2==A3)

#DataFrame(sorted(Balance_df.TICKER_SYMBOL.unique())).to_csv(file_path+'ticker_list_new_613.csv',index=False)

#===============================================================   
    ##检查529和613文件中ticker异同
# ticker_529=pd.read_csv(file_path+'ticker_list_new.csv')
# ticker_613=pd.read_csv(file_path+'ticker_list_new_613.csv')
# [x for x in ticker_613.values if x not in ticker_529.values]

In [ ]:
DataFrame(Income_df.TICKER_SYMBOL.unique()).to_csv(file_path+'ticker_new.csv',index=False)